Membaca data

In [ ]:
import pandas as pd
data = pd.read_csv ('data.csv',
                    usecols=[3],
                    engine='python',
                    delimiter= ",",
                    thousands=","))
data = data.values
data = data.astype('float32')

Pemisahan data

In [ ]:
import numpy as np
x = data
windowSize = 5
def splitData(x, windowSize, a, b, c):
    if ((a+b+c) != 1):
        print("pemisahan tidak valid")
    else:
        train = []
        vald = []
        test = []
        train_size = int(len(x)*a)
        val_size = int(len(x)*b)
        test_size = len(x) - train_size - val_size
        train = x[0:train_size-1,:]
        vald = x[train_size-windowSize-
                1:train_size+val_size-1]
        test = x[train_size+val_size-windowSize-
                1:len(x),:]
        print(len(train), len(vald), len(test))
    return np.array(train), np.array(vald), np.array(test)

train, vald, test = splitData(x, windowSize, 0.6, 0.15, 0.25)

Normalisasi Data

In [ ]:
import numpy as np
def normalize(x,scale):
    normalized = []
    for i in range(len(x)) :
        a = (min(scale))+(x[i]-min(x))
                *(max(scale)-min(scale))/(max(x)-min(x))
        normalized.append(a)
    return np.array(normalized)
scale = (-1,1)
normalized = normalize(data,scale)

Pembuatan dataset

In [ ]:
def createDataset(x, windowSize):
    dataX, dataY = [],[]
    for i in range(len(x)-windowSize):
        a = []
        for j in range(i, i+windowSize):
            for k in range(len(x[j])):
                a.append(x[j,k]);        
        dataX.append(a)
        dataY.append(x[i+windowSize,0])
    return np.array(dataX), np.array(dataY)

trainX, trainY = createDataset(train,windowSize)
valdX, valdY = createDataset(vald, windowSize)
testX, testY = createDataset(test, windowSize)

trainY = np.reshape(trainY,(len(trainY), 1))
valdY = np.reshape(valdY,(len(valdY), 1))
testY = np.reshape(testY,(len(testY), 1))

# analisis Univariat

Memilih data latih, validasi, dan uji 

In [ ]:
trainNorm = normalize(train, scale)
valNorm = normalize(vald, scale)
testNorm = normalize(test, scale)

latih = trainNorm
valid = valNorm
uji = testNorm

# Analisis mulivariate

moving average

In [ ]:
def movingAverage(x,n):
    MA = []
    for i in range(len(x)-n+1):
        a = np.mean(x[i:i+n,])
        MA.append(a)
    return np.arrat(MA)

maSize = 1

maTrain = movingAverage(train,maSize)
maVal = movingAverage(vald,maSize)
maTest = movingAverage(tes,maSize)

def createMA(x,y, maSize):
    y = np.reshape(y, (len(y), 1))
    dataAndNormalized = x[(maSize)-1:len(x)]
    combined = np.concatenate(((dataandNormalized), (y)), 
                             axis=1)
    returned combined

maTrainNorm = normalize(maTrain, scale)
maValNorm = normalize(maVal, scale)
maTestNorm = normalize(maTest, scale)

latih = createMA(trainNorm, maTrainNorm, maSize)
valid = createMA(valNorm, maValNorm, maSize)
uji = createMA(testNorm, maTestNorm, maSize)

Masukan 2 nilai tukar mata uang 

In [ ]:
x = data
x2 = data2

train, vald, test = splitData(x, windowSize, 0.6, 0.15, 0.25)
train2, vald2, test2 = splitData(x2, windowSize, 0.6, 0.15, 0.25)

trainNorm = normalize(train, scale)
valNorm = normalize(vald, scale)
testNorm = normalize(test, scale)

train2norm = normalize(train2, scale)
val2norm = normalize(vald2, scale)
test2norm = normalize(test2, scale)

latih = np. concatenate(((trainNorm), (train2norm)), axis=1)
valid = np. concatenate(((valNorm), (val2norm)), axis=1)
uji = np. concatenate(((testNorm), (test2norm)), axis=1)

# IMPLEMENTASI JST

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.callbacks import ModelCheckpoint

In [ ]:
def updateBobot(x):
    x[0:len(x)-1]=x[0:len(x)-1]*0.
    x[-1]=1.
    return x

savedLoss = []
neuron = 1
nmb_epoch =100
start_time = time.time()
np.random.seed(15)
model = Sequential()
model.add(LSTM(neuron,
              input_dim=windowSize*len(latih.transpose())))
lapisan = Dense(1, trainable=False)
model.add(lapisan)
filepath="bobot\weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=2, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
model.compile(loss='mse', optimizer='RMSprop', stateful=True)
oldBobot = model.get_weights()
updateBobot(oldBobot[3])
model.set_weights(oldBobot)
hist = model.fit(trainX, trainY, nb_epoch=nmb_epoch,
                batch_size=1, verbose=1, callbacks=callbacks_list,
                validation_data=(valdX, valdY))
hist
savedLoss[] = hist.history
print ("training", neuron, "done in")

savedLoss[] = hist.history

Penerapan model

In [ ]:
testPredict = model.predict(testX)
valPredict = model.predict(valdX)

Denormalize

In [ ]:
def denormalize(x, source, scale):
    denormalized = []
    fro i in range(len(x)):
        a - ((x[i]-min(scale))*(max(source)-min(source)))/
        (max(scale))+min(source)
        denormalized.append(a)
    return np.array(denormalized)

denormTestPredict = denormalize(testPredict, test, scale)
denormValPredict = denormalize(valPredict, vald, scale)

# pengujian

membuat tabel

In [ ]:
testLabel = denormalized(testY, test, scale)
valLabel = denormalized(valY, vald, scale)

MAE

In [ ]:
def mae(x,y):
    mae = []
    for i in range(len(x)):
        a = abs(x[i]-y[i])
        mae.append(a)
    return float((sum(mse))/len(y))

RMSE

In [ ]:
def rmse(x,y):
    rmse = []
    for i in range(len(x)):
        a = (x[i]-y[i])**2
        mse.append(a)
return float((sum(mse)/len(y))**0.5)

MAPE

In [ ]:
def mape(x,y):
    mape = []
    for i in range(len(y)):
        a = abs(y[i]-x[i])/y[i]
        mape.append(a)
    return float(sum(mape)/len(y)*100)

Dstat

In [ ]:
def dstat(x,y):
    dstat = 0
    n = len(y)
    for i in range(n-1):
        if(((x[i+1]-y[i])*(y[i+1]-y[i]))>=0):
            dstat += 1
    Dstat = (1/float(n-2))*float(dstat)*100
    return float(Dstat)

--------------------------------------------------------Pembahasan----------------

In [ ]:
scoring[] = (neuron,
            mae (denormValPredict, valLabel),
            rmse (denormValPredict, valLabel),
             mape (denormValPredict, valLabel),
             dstat (denormValPredict, valLabel),
             mae (denormTestPredict, testLabel),
             rmse (denormTestPredict, testLabel),
             mape (denormTestPredict, testLabel),
             dstat (denormTestPredict, testLabel))